## סדר פעולות:

העלאת טבלאות

סידור שמות השדות באנגלית
טבלת מוסדות אחודה- ID, src
בקרה על כפילויות מאותו מקור
סימון בתי ספר עם הפרשים גדולים בין המקורות
מחיקת רשומות ממקור לא מועדף
בניית מקדם מספר כיתות מתוך תלמידים.
החלת המקדם על כל בתי הספר, עיגול מספר התלמידים

מיקום
דאטה גוב
ירושלים- עירייה
בית שמש- עירייה בעזרת סקריפט גאוקודינג
גוגל בעזרת כלים אוטומטיים
יצירת רשימת מוסדות ללא מיקום
חיבור נתונים בעזרת merge
ייצוא לעיגון ידני
העלאת שכבת מוסדות מעיגון ידני
חיבור שכבת העיגון הידני לשאר המוסדות
הוספת שם השכונה

בונוס- תעסוקה
יצירת מקדם מועסקים לפי נתוני למס מעיבוד מיוחד
החלת המקדם, עיגול מספר המועסקים

ייצוא
שכבת נקודות
שכבת אזורי תנועה
סטטיסטיקות מעניינות
טבלת מוסדות עם פער גדול בנתונים


## ייבוא


### ספריות


In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import os
import sys
import math
import warnings
from geopy.geocoders import Nominatim
import pyproj

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### העלת משתנים להרצת הקוד


In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)


### פונקציות גלובליות


In [3]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

### הגדרת מיקום הפלט


In [4]:
output_folder_location=get_output_folder_location(r'{}\background_files\output_folder_location.txt'.format(cwd))
out_shp = r'{}\education.shp'.format(output_folder_location)

## העלאת טבלאות


In [5]:
#בתי ספר וגנים מעיריית בית שמש
BShemesh=up_load_df(r'{}\background_files\betshemesh_muni\חינוך'.format(cwd),'מצבת מוסדות חינוך תשפג משרדי')
BShemesh = remove_spaces_in_columns(BShemesh)

In [6]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(cwd),'מוסדות בירושלים 2020')
JLM = remove_spaces_in_columns(JLM)

In [7]:
#גנים ממשרד החינוך בכל הארץ
GKgarden=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(cwd),'kgarden_2023')
GKgarden = remove_spaces_in_columns(GKgarden)

In [8]:
#בתי ספר ממשרד החינוך בכל הארץ
Gschool=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(cwd),'schools_2023')
Gschool = remove_spaces_in_columns(Gschool)

In [9]:
#יישובים בשטח צתאל שם וקוד יישוב
JTMT_setls=up_load_df(r'{}\background_files'.format(cwd),'210615_מקוצר_רשימת_יישובים_באחריות_צתאל')
JTMT_setls = remove_spaces_in_columns(JTMT_setls)

In [10]:
#קאורדינטות של מוסדות חינוך בכל הארץ ממשרד החינוך
GXY=up_load_df(r'{}\background_files\ministry_of_education\מוסדות'.format(cwd),'moe_mosdot_coordinates_2022')
GXY = remove_spaces_in_columns(GXY)

In [11]:
#מספרי מוסדות, כיתות, ותלמידים בכל רשות מתוך קובץ יישובים של למ"ס
CBS_setl=up_load_df(r'{}\background_files\cbs'.format(cwd),'CBS_EDU_yishuv_2021')
CBS_setl = remove_spaces_in_columns(CBS_setl)

In [12]:
# חיבור שתי הטבלאות לפי עמודת 'סמל מוסד'
df_dropped = Gschool.drop(columns=['דוא"ל_מזכירות'])
institutions_table = pd.merge(Gschool, GKgarden, on='סמל_מוסד', how='outer')

In [13]:
# רשימת כל העמודות בטבלה
columns = institutions_table.columns

# מציאת כל העמודות שמסתיימות ב-_x
for col in columns:
    if col.endswith('_x'):
        # יצירת שם העמודה המקביל עם _y
        col_y = col.replace('_x', '_y')
        
        # בדיקת קיום של העמודה המקבילה עם _y
        if col_y in columns:
            # יצירת עמודה חדשה עם הערכים המאוחדים
            institutions_table[col.replace('_x', '')] = institutions_table[col].combine_first(institutions_table[col_y])
            
            # מחיקת העמודות הכפולות _x ו-_y
            institutions_table = institutions_table.drop(columns=[col, col_y])

# הצגת הטבלה לאחר האיחוד
institutions_table.to_excel('ministry_of_education_institutions_merged.xlsx')

institutions_table.head()

,סמל_מוסד,"דוא""ל_מזכירות",שם_מוסד,יישוב,כתובת,"כתובת_חט""ב",טלפון,סוג_חינוך,מעמד_משפטי,סוג_פיקוח,מגזר,יחידת_דיווח,שכבה,מספר_תלמידים
0,110023,brahatok@hinuchm.k12.il,בית יעקב מטרסדורף,ירושלים,זכרון יעקב 3,NaN,02-5372853,רגיל,מוכר שאיננו רשמי,חרדי,יהודי,רשת החינוך העצמאי,א - ח,782.00
1,110098,NaN,השחר,טל שחר,טל שחר,NaN,08-9349345,רגיל,רשמי,ממלכתי,יהודי,ירושלים,א - ו,420.00
2,110106,carmitbar6@hinuchm.k12.il,אגרון גרשון,ירושלים,זנגויל 29,NaN,02-6411444,רגיל,רשמי,ממלכתי,יהודי,מנח'י,גן - ו,243.00
3,110130,brahalan@hinuchm.k12.il,סאלד,ירושלים,כט' בנובמבר 6,NaN,02-5633675,רגיל,רשמי,ממלכתי,יהודי,מנח'י,א - ו,328.00
4,110155,bateljos@hinuchm.k12.il,בית הכרם,ירושלים,מעגל בית המדרש 8,NaN,02-6512155,רגיל,רשמי,ממלכתי,יהודי,מנח'י,א - ו,376.00


In [ ]:
institutions_table=institutions_table.merge(JLM, how='outer', left_on='סמל_מוסד', right_on='סמל_חינוך', )
institutions_table.head()

In [14]:
institutions_table=institutions_table.merge(BShemesh, on='סמל_מוסד', how='outer')
# BShemesh.head()
institutions_table.head()
# institutions_table.to_excel('institutions_table.xlsx')

In [17]:
# רשימת כל העמודות בטבלה
columns = institutions_table.columns

# מציאת כל העמודות שמסתיימות ב-_x
for col in columns:
    if col.endswith('_x'):
        # יצירת שם העמודה המקביל עם _y
        col_y = col.replace('_x', '_y')
        
        # בדיקת קיום של העמודה המקבילה עם _y
        if col_y in columns:
            # יצירת עמודה חדשה עם הערכים המאוחדים
            institutions_table[col.replace('_x', '')] = institutions_table[col].combine_first(institutions_table[col_y])
            
            # מחיקת העמודות הכפולות _x ו-_y
            institutions_table = institutions_table.drop(columns=[col, col_y])

# הצגת הטבלה לאחר האיחוד
# מילוי הערכים הריקים בעמודה 'סמל_מוסד' עם הערכים מעמודה 'סמל_חינוך'
institutions_table['סמל_מוסד'] = institutions_table['סמל_מוסד'].fillna(institutions_table['סמל_חינוך'])

# הצגת הטבלה לאחר השינוי
institutions_table.to_excel('institutions_table_1.xlsx')


In [ ]:
institutions_table.head()

## שדות


ID
address
sector
kgarden_stu
ele_stu
mid_stu
high_stu
kgarden_class
ele_class
mid_class
high_class
'x'
'y'
src


#### פונקציות


In [12]:
def age_distribute(df):
    #ages definition
    ages = {'kgarden':[3, 4, 5],'ele':['א', 'ב', 'ג', 'ד','ה','ו'],'mid':['ז', 'ח', 'ט'], 'high':['י', 'יא', 'יב', 'יג', 'יד']}
    ages_number ={'גן':1,'א': 1,'ב': 2,'ג': 3,'ד': 4,'ה': 5,'ו': 6,'ז': 7,
        'ח': 8,'ט': 9,'יד': 12.5,'יג': 12.3,'יב': 12,'יא': 11,'י': 10}
    for i in ages_number:
        df.loc[df['bottom_age']==i,'b_age_num']=ages_number[i]
        df.loc[df['top_age']==i,'t_age_num']=ages_number[i]

    ##calculate new columns

    #kgarden student
    df.loc[df['top_age'].isin(ages['kgarden']),'kgarden_stu'] = df['students_total']
    if not len(df.loc[df['bottom_age'].isin(ages['kgarden']) & ~df['top_age'].isin(ages['kgarden'])])==0:
        print('need to write code for situation of kgarden and school in the same institution')

    #high school fixing where it's 0. 
    #assumes that every age has equal number of student (exept of 13 and 14 class which 0.3 and 0.2 of the other classes)

    #all ages are high school
    df.loc[(df['b_age_num']>9) & (df['high_stu']==0),'high_stu'] =df['students_total'].round()

    #ages are middle and high school and there is a middle school students
    df.loc[(df['t_age_num']>9) &(df['b_age_num']>6) & (df['high_stu']==0)& (df['mid_stu']!=0),'high_stu'] =\
    (df['students_total']- df['mid_stu'])

    #ages are elementary, middle and high school and there is a middle school ages
    df.loc[(df['t_age_num']>9) &(df['b_age_num']<7) & (df['high_stu']==0)& (df['mid_stu']!=0),'high_stu'] =\
    ((df['students_total']- df['mid_stu'])*((df['t_age_num']-9)/(df['t_age_num']-df['b_age_num']-2))).round()

    #ages are middle and high school or diverse all school ages
    df.loc[(df['t_age_num']>9) &(df['b_age_num']<10) & (df['high_stu']==0)& (df['mid_stu']==0),'high_stu'] =\
    (df['students_total']*((df['t_age_num']-9)/(df['t_age_num']-df['b_age_num']+1))).round()



    #middle school fix where it's 0. 
    #assumes that every age has equal number of student (exept of 13 and 14 class which 0.3 and 0.2 of the other classes)

    #all ages are middle school
    df.loc[(df['t_age_num']<10) & (df['b_age_num']>6) & (df['mid_stu']==0),'mid_stu'] =df['students_total'].round()

    #ages are middle and high school
    df.loc[(df['b_age_num']<10) &(df['b_age_num']>6) & (df['mid_stu']==0),'mid_stu'] =\
    (df['students_total']- df['high_stu'])

    #ages are middle and elementary school
    df.loc[(df['t_age_num']>6) &(df['t_age_num']<10) & (df['mid_stu']==0),'mid_stu'] =\
    df['students_total']*((df['t_age_num']-6)/(df['t_age_num']-df['b_age_num']+1)).round()

    #diverse all school ages
    df.loc[(df['t_age_num']>6) &(df['b_age_num']<7) & (df['mid_stu']==0),'mid_stu'] =\
    df['students_total']*(3/(df['t_age_num']-df['b_age_num']+1)).round()

    #calculate elementary student
    df.loc[(df['b_age_num']<7),'ele_stu']=df['students_total']-df['high_stu']-df['mid_stu'].round()

    #checking student numbers
    if len(df.loc[df['students_total']!=df['high_stu']+df['mid_stu']+df['ele_stu']+df['kgarden_stu']]):
        warnings.warn(f"Warning: there is a problem with age distribution. total studens is different fof sum of them")

    return df

In [13]:
# שינוי שמות עמודות לתיאום בין הטבלאות
GXY.rename(columns={'SEMEL_MOSAD': 'סמל_מוסד', 'ITM_X': 'X', 'ITM_Y': 'Y'}, inplace=True)

# ביצוע מיזוג בין הטבלאות על סמך עמודת "סמל_מוסד"
Gschool=Gschool.merge(GXY[['סמל_מוסד', 'X', 'Y']], on='סמל_מוסד', how='left')
GKgarden=GKgarden.merge(GXY[['סמל_מוסד', 'X', 'Y']], on='סמל_מוסד', how='left')


ministry_of_education_students=pd.concat([Gschool, GKgarden])
# # סינון מוסדות על סמך היישובים הנמצאים בצתאל
# Gschool=Gschool[Gschool['יישוב'].isin(JTMT_setls['שם_יישוב'])]
# GKgarden=GKgarden[GKgarden['יישוב'].isin(JTMT_setls['שם_יישוב'])]

ministry_of_education_students=ministry_of_education_students[ministry_of_education_students['יישוב'].isin(JTMT_setls['שם_יישוב'])]
# GKgarden.columns = ['ID', 'name', 'settlement', 'address','a','phone','type','legal_status',
#        'supervision_type', 'sector', 'יחידת_דיווח', 'age', 'kgarden_stu','x','y']
# Gschool.columns = ['ID', 'name', 'settlement', 'address','כתובת חט"ב', 'טלפון', 'דוא"ל_מזכירות',
#        'סוג_חינוך', 'legal_status','supervision_type', 'sector', 'יחידת_דיווח', 'ages', 'students_total','x','y']

# #calculate columns
# GKgarden[['ele_stu','mid_stu','high_stu','ele_class','mid_class','high_class','x','y']] = 0
# GKgarden['kgarden_class'] = 1

# Gschool[['bottom_age', 'top_age']] = Gschool['ages'].str.split(' - ',  expand=True)
# Gschool['students_total'] = Gschool['students_total'].fillna(0)
# Gschool[['kgarden_stu','ele_stu','mid_stu','high_stu','kgarden_class','ele_class','mid_class','high_class','x','y']] = 0

# # Gschool = age_distribute(Gschool)

In [ ]:
# יצירת רשימת העמודות
cols = ['גן1', 'גן2', 'גן3', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']
for cols in cols:
    ministry_of_education_students[cols] = 0  # מוסיפים עמודות חדשות ומאפסים אותן

# הגדרת מילון טווחים שממפה כל טווח לרשימת השכבות המתאימות
division = {
    'א - ח': ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח'],
    'א - ו': ['א', 'ב', 'ג', 'ד', 'ה', 'ו'],
    'גן - ו': ['גן1', 'גן2', 'גן3', 'א', 'ב', 'ג', 'ד', 'ה', 'ו'],
    'ד - ח': ['ד', 'ה', 'ו', 'ז', 'ח'],
    'א - ג': ['א', 'ב', 'ג'],
    'א - ד': ['א', 'ב', 'ג', 'ד'],
    'ז - ח': ['ז', 'ח'],
    'א - ז': ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז'],
    'ב - ח': ['ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח'],
    'גן - יב': ['גן1', 'גן2', 'גן3', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב'],
    'א - יב': ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב'],
    'א - ט': ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט'],
    'ז - ט': ['ז', 'ח', 'ט'],
    'ז - יב': ['ז', 'ח', 'ט', 'י', 'יא', 'יב'],
    'ט - יב': ['ט', 'י', 'יא', 'יב'],
    'ז - יד': ['ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד'],
    'ט - יג': ['ט', 'י', 'יא', 'יב', 'יג'],
    'ט - יד': ['ט', 'י', 'יא', 'יב', 'יג', 'יד'],
    'י - יב': ['י', 'יא', 'יב'],
    'יב - יב': ['יב'],
    'ט - יא': ['ט', 'י', 'יא'],
    'ז - יג': ['ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג'],
    'י - יג': ['י', 'יא', 'יב', 'יג'],
    'יג - יד': ['יג', 'יד'],
    'י - יד': ['י', 'יא', 'יב', 'יג', 'יד'],
    'ח - יב': ['ח', 'ט', 'י', 'יא', 'יב'],
    'ו - ח': ['ו', 'ז', 'ח'],
    'ו - יב': ['ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב'],
    'גן - ח': ['גן1', 'גן2', 'גן3', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח'],
    'ה - ח': ['ה', 'ו', 'ז', 'ח'],
    'ז - יא': ['ז', 'ח', 'ט', 'י', 'יא'],
    'ט - ט': ['ט'],
    'יא - יב': ['יא', 'יב'],
    'א - יד': ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד'],
    'ז - י': ['ז', 'ח', 'ט', 'י'],
    'ט - י': ['ט', 'י'],
    'א - ב': ['א', 'ב'],
    'ו - יד': ['ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד'],
    'א - ה': ['א', 'ב', 'ג', 'ד', 'ה'],
    'ח - יא': ['ח', 'ט', 'י', 'יא'],
    'י - יא': ['י', 'יא'],
    'יג - יג': ['יג'],
    'ה - ז': ['ה', 'ו', 'ז'],
    'יא - יא': ['יא'],
    'ד - ו': ['ד', 'ה', 'ו'],
    'א - י': ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י'],
    'ג - ח': ['ג', 'ד', 'ה', 'ו', 'ז', 'ח'],
    'גן - גן': ['גן1', 'גן2', 'גן3']
}

# פונקציה לחלק את מספר התלמידים בעמודות המתאימות
def division_students(row):
    layer_range = division.get(row['שכבה'], [])
    if not layer_range:
        return row  # אין התאמה לטווח השכבה, מחזירים את השורה כמו שהיא
    
    # חלוקת מספר התלמידים בעמודות המתאימות
    number_of_columns = len(layer_range)
    students_per_column = row['מספר_תלמידים'] / number_of_columns

    for col in layer_range:
        row[col] = students_per_column
    
    return row

# החלת הפונקציה על כל שורה ב-DataFrame
ministry_of_education_students = ministry_of_education_students.apply(division_students, axis=1)

# הצגת התוצאות
ministry_of_education_students.head()

In [ ]:
# Gschool['settlement']['ירושלים']
JLM=Gschool[Gschool['settlement']=='ירושלים']
JLM

JLM['classes_total']=JLM['kgarden_class'].sum()+JLM['ele_class'].sum()+JLM['mid_class'].sum()+JLM['high_class'].sum()
average_student_to_class_in_jlm = JLM['students_total'].sum()/JLM['classes_total'].sum()
average_student_to_class_in_jlm

In [ ]:
average_student_to_class_in_jlm = JLM['students_total'].sum()/JLM['classes_total'].sum()


In [ ]:
#classes by age from the average in jerusalem
for age in ['kgarden','ele','mid','high']:
    Gschool[age+'_class'] = (Gschool[age+'_stu'] / average_student_to_class_in_jlm).apply(lambda x: math.ceil(x))

#sector
Gschool.loc[Gschool['sector']!='יהודי','sector'] = 1
Gschool.loc[Gschool['sector']=='יהודי','sector'] = 3
Gschool.loc[Gschool['supervision_type']=='חרדי','sector'] = 2

GKgarden.loc[GKgarden['sector']!='יהודי','sector'] = 1
GKgarden.loc[GKgarden['sector']=='יהודי','sector'] = 3
GKgarden.loc[GKgarden['supervision_type']=='חרדי','sector'] = 2

#src
Gschool['src'] = 'education ministry'
GKgarden['src'] = 'education ministry'

#erase Irelevant columns
Gschool = Gschool[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]
GKgarden = GKgarden[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]

In [ ]:
#change hebrew
GKgarden.columns = ['ID', 'name', 'settlement', 'address','a','phone','type','legal_status',
       'supervision_type', 'sector', 'יחידת_דיווח', 'age', 'kgarden_stu']
Gschool.columns = ['ID', 'name', 'settlement', 'address','כתובת חט"ב', 'טלפון', 'דוא"ל_מזכירות',
       'סוג_חינוך', 'legal_status','supervision_type', 'sector', 'יחידת_דיווח', 'ages', 'students_total']

#take only in JTMT settlements
setls = list(JTMT_setls['שם_יישוב'])
GKgarden = GKgarden.loc[GKgarden['settlement'].isin(setls)]

Gschool = Gschool.loc[Gschool['settlement'].isin(setls)]

#calculate columns
GKgarden[['ele_stu','mid_stu','high_stu','ele_class','mid_class','high_class','x','y']] = 0
GKgarden['kgarden_class'] = 1

Gschool[['bottom_age', 'top_age']] = Gschool['ages'].str.split(' - ',  expand=True)
Gschool['students_total'] = Gschool['students_total'].fillna(0)
Gschool[['kgarden_stu','ele_stu','mid_stu','high_stu','kgarden_class','ele_class','mid_class','high_class','x','y']] = 0

Gschool = age_distribute(Gschool)

#classes
#classes by age from the average in jerusalem
for age in ['kgarden','ele','mid','high']:
    Gschool[age+'_class'] = (Gschool[age+'_stu'] / average_student_to_class_in_jlm).apply(lambda x: math.ceil(x))

#sector
Gschool.loc[Gschool['sector']!='יהודי','sector'] = 1
Gschool.loc[Gschool['sector']=='יהודי','sector'] = 3
Gschool.loc[Gschool['supervision_type']=='חרדי','sector'] = 2

GKgarden.loc[GKgarden['sector']!='יהודי','sector'] = 1
GKgarden.loc[GKgarden['sector']=='יהודי','sector'] = 3
GKgarden.loc[GKgarden['supervision_type']=='חרדי','sector'] = 2

#src
Gschool['src'] = 'education ministry'
GKgarden['src'] = 'education ministry'

#erase Irelevant columns
Gschool = Gschool[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]
GKgarden = GKgarden[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]

#### עיריית בית שמש


In [13]:
#change hebrew
BShemesh.columns = ['sector', 'name', 'ID', 'neighbourhood', 'address', 'b2', 'a1',
       'a5', 'a8', 'a7', 'a6', 'a4', 'a3',
       'a9', 'a11', 'a10', 'Kgarden4', 'Kgarden5',
       'SpeKgarden', 'Kgarden_total', '1_1', '1_2', '1_3', '1_4', '1_5', '1_', '2_1', '2_2', '2_3', '2_4', '2_', '3_1', '3_2', '3_3', '3_4', '3_', '4_1', '4_2', '4_3', '4_4', '4_', '5_1', '5_2', '5_3', '5_4', '5_', '6_1', '6_2', '6_3', '6_4', '6_', 'a20', '7_1', '7_2', '7_3', '7_4', '7_', '8_1', '8_2', '8_3', '8_4', '8_', '', '9_1', '9_2', '9_3', '9_', '10_1', '10_2', '10_3', '10_', '11_1', '11_2', '11_3', '11_', '12_1', '12_2', '12_3', '12_', 'a22', 'a21', 'X', 'Y']

#calculate columns
#student number
BShemesh['kgarden_stu'] = BShemesh['Kgarden_total'] 
BShemesh['ele_stu'] = BShemesh[['1_1', '1_2', '1_3', '1_4', '1_5',
                            '2_1', '2_2', '2_3', '2_4',
                            '3_1', '3_2', '3_3', '3_4',
                            '4_1', '4_2', '4_3', '4_4',
                            '5_1', '5_2', '5_3', '5_4',
                            '6_1', '6_2', '6_3', '6_4',
                            '1_', '2_', '3_', '4_','5_', '6_']].sum(axis=1)

BShemesh['mid_stu'] = BShemesh[['6_1', '6_2', '6_3', '6_4',
                            '7_1', '7_2', '7_3', '7_4',
                            '8_1', '8_2', '8_3', '8_4',
                            '9_1', '9_2', '9_3',
                            '7_', '8_','9_']].sum(axis=1)
BShemesh['high_stu'] = BShemesh[['10_1', '10_2', '10_3',
                            '11_1', '11_2', '11_3',
                            '12_1', '12_2', '12_3',
                            '10_', '11_', '12_']].sum(axis=1)

BShemesh['kgarden_class'] = BShemesh[['Kgarden4', 'Kgarden5']].count(axis=1)
#classes
BShemesh['ele_class'] = BShemesh[['1_1', '1_2', '1_3', '1_4', '1_5',
                            '2_1', '2_2', '2_3', '2_4',
                            '3_1', '3_2', '3_3', '3_4',
                            '4_1', '4_2', '4_3', '4_4',
                            '5_1', '5_2', '5_3', '5_4',
                            '6_1', '6_2', '6_3', '6_4',
                            '1_', '2_', '3_', '4_','5_', '6_']].count(axis=1)

BShemesh['mid_class'] = BShemesh[['6_1', '6_2', '6_3', '6_4',
                            '7_1', '7_2', '7_3', '7_4',
                            '8_1', '8_2', '8_3', '8_4',
                            '9_1', '9_2', '9_3',
                            '7_', '8_','9_']].count(axis=1)
BShemesh['high_class'] = BShemesh[['10_1', '10_2', '10_3',
                            '11_1', '11_2', '11_3',
                            '12_1', '12_2', '12_3',
                            '10_', '11_', '12_']].count(axis=1)
#sector
BShemesh.loc[BShemesh['sector']=='חרדי','sector'] = 2
BShemesh.loc[BShemesh['sector']=='ממ"ח','sector'] = 2
BShemesh.loc[BShemesh['sector']=='ממ"ד','sector'] = 3
BShemesh.loc[BShemesh['sector']=='מ"מ','sector'] = 3
BShemesh.loc[BShemesh['sector']=='עירוני','sector'] = 3

#adding another columns
BShemesh['settlement'] = 'בית שמש'
BShemesh[['x', 'y']] = 0

#src
BShemesh['src'] = 'Beit Shemesh muni'

#erase Irelevant columns
BShemesh = BShemesh[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]

#### עיריית ירושלים


In [297]:
#change hebrew
JLM.columns = ['muni_ID', 'ID', 'name', 'sector', 'פיקוח', 'שלב_חינוך',
       'is_speciel', 'מעמד_משפטי', 'bottom_age', 'top_age', 'classes_total',
       'mid_stu', 'high_stu', 'students_total',
       'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'x', 'y',
       'Unnamed:_18', 'Unnamed:_19'] 

#adding new columns
JLM['settlement'] = 'ירושלים'
JLM[['address','kgarden_stu','ele_stu','kgarden_class', 'ele_class', 'mid_class', 'high_class']] = 0
JLM[['kgarden_stu','high_stu','mid_stu']] = JLM[['kgarden_stu','high_stu','mid_stu']].fillna(0)

#calculate student number
JLM = age_distribute(JLM)

#classes by age
JLM['stu_to_class'] = JLM['students_total']/JLM['classes_total']
for age in ['kgarden','ele','mid','high']:
    JLM[age+'_class'] = (JLM[age+'_stu'] / JLM['stu_to_class']).round()

average_student_to_class_in_jlm = JLM['students_total'].sum()/JLM['classes_total'].sum()
    
#sector
JLM.loc[JLM['sector']=='ערבי','sector'] = 1
JLM.loc[JLM['sector']=='חרדי','sector'] = 2
JLM.loc[JLM['sector']=='כללי','sector'] = 3

#src
JLM['src'] = 'Jerusalem muni'

#erase Irelevant columns
JLM = JLM[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]

#### משרד החינוך


In [298]:
#change hebrew
GKgarden.columns = ['ID', 'name', 'settlement', 'address','a','phone','type','legal_status',
       'supervision_type', 'sector', 'יחידת_דיווח', 'age', 'kgarden_stu']
Gschool.columns = ['ID', 'name', 'settlement', 'address','כתובת חט"ב', 'טלפון', 'דוא"ל_מזכירות',
       'סוג_חינוך', 'legal_status','supervision_type', 'sector', 'יחידת_דיווח', 'ages', 'students_total']

#take only in JTMT settlements
setls = list(JTMT_setls['שם_יישוב'])
GKgarden = GKgarden.loc[GKgarden['settlement'].isin(setls)]

Gschool = Gschool.loc[Gschool['settlement'].isin(setls)]

#calculate columns
GKgarden[['ele_stu','mid_stu','high_stu','ele_class','mid_class','high_class','x','y']] = 0
GKgarden['kgarden_class'] = 1

Gschool[['bottom_age', 'top_age']] = Gschool['ages'].str.split(' - ',  expand=True)
Gschool['students_total'] = Gschool['students_total'].fillna(0)
Gschool[['kgarden_stu','ele_stu','mid_stu','high_stu','kgarden_class','ele_class','mid_class','high_class','x','y']] = 0

Gschool = age_distribute(Gschool)

#classes
#classes by age from the average in jerusalem
for age in ['kgarden','ele','mid','high']:
    Gschool[age+'_class'] = (Gschool[age+'_stu'] / average_student_to_class_in_jlm).apply(lambda x: math.ceil(x))

#sector
Gschool.loc[Gschool['sector']!='יהודי','sector'] = 1
Gschool.loc[Gschool['sector']=='יהודי','sector'] = 3
Gschool.loc[Gschool['supervision_type']=='חרדי','sector'] = 2

GKgarden.loc[GKgarden['sector']!='יהודי','sector'] = 1
GKgarden.loc[GKgarden['sector']=='יהודי','sector'] = 3
GKgarden.loc[GKgarden['supervision_type']=='חרדי','sector'] = 2

#src
Gschool['src'] = 'education ministry'
GKgarden['src'] = 'education ministry'

#erase Irelevant columns
Gschool = Gschool[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]
GKgarden = GKgarden[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]

#### למס


In [299]:
CBS_setl.columns = ['set_ID', 'settlement',
       'kgarden_stu',
       'ילדים_בגנים_של_משרד_החינוך_גיל_3_תשפ"א_2020/21',
       'ילדים_בגנים_של_משרד_החינוך_גיל_4_תשפ"א_2020/21',
       'ילדים_בגנים_של_משרד_החינוך_גיל_5_תשפ"א_2020/21',
       'ילדים_בגנים_של_משרד_החינוך_גיל_6_תשפ"א_2020/21',
       'schools',
       'ele_schools',
       'בתי_ספר_על-יסודיים_תשפ"א_2020/21',
       'mid_school', 'high_school',
       'כיתות_סה"כ_תשפ"א_2020/21', 'ele_class',
       'כיתות_בבתי_ספר_על-יסודיים_תשפ"א_2020/21',
       'mid_class',
       'high_class', 'תלמידים_סה"כ_תשפ"א_2020/21',
       'ele_stu',
       'תלמידים_בבתי_ספר_על-יסודיים_תשפ"א_2020/21',
       'mid_stu',
       'high_stu',
       'ממוצע_תלמידים_לכיתה_בבתי_ספר_יסודיים_תשפ"א_2020/21',
       'אחוז_תלמידים_נושרים_סה"כ_תשפ"א_2020/21',
       'אחוז_תלמידים_נושרים_בנים_תשפ"א_2020/21',
       'אחוז_תלמידים_נושרים_בנות_תשפ"א_2020/21',
       'אחוז_זכאים_לתעודת_בגרות_מבין_תלמידי_כיתות_יב_תשפ"א_2020/21',
       'אחוז_זכאים_לתעודת_בגרות_שעמדו_בדרישות_הסף_של_האוניברסיטאות_מבין_תלמידי_כיתות_יב_תשפ"א_2020/21',
       'השכלה_גבוהה_אחוז_בעלי_תארים_מישראל_מתוך_אוכלוסיית_בני_55-35_תשפ"ב_2021/22',
       'השכלה_גבוהה_אחוז_הנכנסים_להשכלה_גבוהה_בתוך_8_שנים_בקרב_תלמידי_יב_תשפ"ב_2021/22',
       'השכלה_גבוהה_אחוז_סטודנטים_מתוך_אוכלוסיית_בני_25-20_תשפ"ב_2021/22',
       'השכלה_גבוהה_אחוז_סטודנטים_מתוך_כלל_האוכלוסייה_תשפ"ב_2021/22',
       'עובדי_הוראה_מספר_עובדי_הוראה', 'עובדי_הוראה_ממוצע_שעות_עבודה_לשבוע',
       'עובדי_הוראה_אחוז_עובדי_הוראה_חדשים',
       'עובדי_הוראה_אחוז_שעות_היעדרות_מסך_שעות_עבודה_שנתיות',
       'עובדי_הוראה_ממוצע_שנות_ותק_מוכרות_בהוראה',
       'עובדי_הוראה_אחוז_בעלי_דרגת_שכר_תואר_שני_ומעלה',
       'עובדי_הוראה_אחוז_עובדי_הוראה_משתלמים_מכלל_עובדי_הוראה',
       'teacher_by_stu',
       'עובדי_הוראה_ממוצע_שעות_עבודה_שבועיות_לתלמיד']

CBS_setl = CBS_setl[['set_ID', 'settlement',
       'kgarden_stu',
       'schools',
       'ele_class',
       'mid_class',
       'high_class', 
       'ele_stu',
       'mid_stu',
       'high_stu']]

CBS_setl.replace('-', 0.001, inplace=True)

## טבלת מוסדות אחודה- ID, src


In [300]:
rows = pd.concat([BShemesh,Gschool,GKgarden,JLM]) #JLM is 2020, the others are 2023

In [301]:
inst = rows.groupby('ID').agg('first').reset_index()

## מיקום


#### datagov from education ministry


In [302]:
inst_xy = inst.merge(GXY[['SEMEL_MOSAD','ITM_X', 'ITM_Y']],how='left',left_on = 'ID', right_on='SEMEL_MOSAD')

inst_xy.loc[inst_xy['ITM_X'].notna(),'x'] = inst_xy['ITM_X']

inst_xy.loc[inst_xy['ITM_Y'].notna(),'y'] = inst_xy['ITM_Y']

inst = inst_xy[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]

#### OSM geocoding


In [31]:
def OSMgeocoding(df):
    geolocator = Nominatim(user_agent="my_geocoder")
    itm = pyproj.Proj(init='epsg:2039')
    for i in df['ID']:
        location = geolocator.geocode(str(df.loc[df['ID'] == i]['f_address'])[4:-31])
        if location:
            x, y = pyproj.transform(pyproj.Proj(init='epsg:4326'), itm, location.longitude, location.latitude) #to ITM
            df.loc[df['ID']==i,'x_osm'] = x
            df.loc[df['ID']==i,'y_osm'] = y
    return df.loc[df['x_osm'].notna()]

In [ ]:
no_xy = inst.loc[(inst['x']==0) | (inst['x'].isna())][['ID','address','settlement']]
# full address
no_xy['f_address'] = no_xy['address'].astype(str).str.cat(no_xy['settlement'], sep=', ').apply(lambda x: f"{x}, ישראל")

#search on OSM
if input('take a lot of time, enter y for do it!\n')=='y':
    inst_xy_osm = OSMgeocoding(no_xy)

In [305]:
#merge to main inst dataframe
inst_xy = inst.merge(inst_xy_osm[['ID','x_osm','y_osm']],how='left',on = 'ID')
inst_xy.loc[(inst['x']==0)| (inst['x'].isna()),'x'] = inst_xy['x_osm']
inst_xy.loc[(inst['y']==0)| (inst['y'].isna()),'y'] = inst_xy['y_osm']
inst = inst_xy[['ID','name','sector','address','settlement','x', 'y','kgarden_stu','ele_stu','mid_stu','high_stu',
                     'kgarden_class','ele_class','mid_class','high_class','src']]

~בית שמש- עירייה בעזרת סקריפט גאוקודינג- 89
~- 298 גוגל בעזרת כלים אוטומטיים
יצירת רשימת מוסדות ללא מיקום


### יצירת GDF


In [306]:
geometry = [Point(xy) for xy in zip(inst['x'], inst['y'])]

In [307]:
inst = gpd.GeoDataFrame(inst, geometry=geometry,crs='EPSG:2039')

## בקרה


### פונקציות


In [257]:
def compare_rows(df1, df2, ID_field, cmp_fields, threshold=0.2):
    ratio_fields = [f+'_ratio' for f in cmp_fields]
    
    # Merge the two DataFrames on a common index or key
    merged_df = pd.merge(df1, df2, on=ID_field, how='inner', suffixes=('_1', '_2'))
    merged_df['isdifferent'] = False
    #calculate ratio to a new column
    for c in cmp_fields:
        merged_df[c+'_ratio'] = merged_df[c+'_1'] / merged_df[c+'_2']
        merged_df.loc[abs(merged_df[c+'_ratio']-1) > threshold,'isdifferent'] = True
    
    return merged_df.loc[merged_df['isdifferent']]

### השוואה ברמת מוסד


#### בית שמש - משרד החינוך


In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0.3
dif = compare_rows(BShemesh, GKgarden,'ID',cmp_setl, threshold=threshold)
len(dif)

In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0.
dif = compare_rows(BShemesh, GKgarden,'ID',cmp_setl, threshold=threshold)
len(dif)

In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0.3
dif = compare_rows(BShemesh, Gschool,'ID',cmp_setl, threshold=threshold)
len(dif)

In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0
dif = compare_rows(BShemesh, GKgarden,'ID',cmp_setl, threshold=threshold)
len(dif)

#### ירושלים- משרד החינוך


In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0.3
dif = compare_rows(JLM, GKgarden,'ID',cmp_setl, threshold=threshold)
len(dif)

In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0
dif = compare_rows(JLM, GKgarden,'ID',cmp_setl, threshold=threshold)
len(dif)

In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0.3
dif = compare_rows(JLM, Gschool,'ID',cmp_setl, threshold=threshold)
len(dif)

In [ ]:
#fields to compare
cmp_setl = ['kgarden_stu',
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0
dif = compare_rows(JLM, Gschool,'ID',cmp_setl, threshold=threshold)
len(dif)

### השוואה ברמת יישוב אל מול נתוני למס


In [ ]:
#sum by settiemnt
settlement = inst.groupby('settlement').agg({'ID':'count', 'kgarden_stu':'sum', 'ele_stu':'sum', 'mid_stu':'sum',
                                'high_stu':'sum', 'kgarden_class':'sum', 'ele_class':'sum',
                                'mid_class':'sum', 'high_class':'sum'}).reset_index()
settlement['schools'] = settlement['ID'] - settlement['kgarden_class']
settlement.replace(0, 0.001, inplace=True)

#fields to compare
cmp_setl = ['kgarden_stu','schools', #<inst_count> - <kgarden_class> 
       'ele_class','mid_class','high_class', 
       'ele_stu','mid_stu','high_stu']
#how much different
threshold = 0.3

different_settlements = compare_rows(settlement, CBS_setl,'settlement',cmp_setl, threshold=threshold)

different_settlements

בקרה על מספר המוסדות - שלא איבדנו מוסד.
בקרה על מספר התלמידים והכיתות הכולל

~סימון בתי ספר עם הפרשים גדולים בין המקורות


## ייצוא


In [ ]:
inst.to_file(out_shp,encoding='utf-8')